In [15]:

import pandas as pd
from sklearn.model_selection import train_test_split as TTS

# data = pd.read_csv(r"C:\Users\96361\Desktop\selflearning\github\Machine-Learning\01 决策树课件数据源码\data.csv",index_col = 0)
data = pd.read_csv(r"C:\Users\96361\iCloudDrive\数据挖掘实践-胡仕成\A\continuousData\Sampledata-continuous.CSV",encoding='GBK')
data["CUSTOMER_TYPE"] = (data["CUSTOMER_TYPE"]== "2G").astype("int")

X_train=data.iloc[:,:7].to_numpy()
y_train=data.iloc[:,7].to_numpy()
print(X_train,type(X_train))

Xtrain, Xtest, Ytrain, Ytest = TTS(X_train,y_train,test_size=0.3,random_state=200)
# print(Xtrain,type(Xtrain))

cases=Xtrain.tolist()
print(Ytrain)

labels = [[0 for col in range(1)] for row in range(19)]
# print(y_train.tolist(),type(y_train))
for i in range(0,len(Ytrain)):
    # print(i)
    labels[i][0]=Ytrain[i]
    
print(cases)
print(labels)



Xtest_extra = [[0.12,	0, 0.72,	0.01, 0.02,	0.02,	0]]
# Ytest_extra = [['2G']]
Ytest_extra = [[1]]

[[0.12 1.   0.99 0.34 0.31 0.25 0.  ]
 [1.   0.   0.29 0.   0.03 0.01 0.  ]
 [0.35 0.   1.   0.36 0.47 0.47 0.03]
 [0.   0.   0.98 0.18 0.19 0.18 0.01]
 [0.55 0.   0.84 0.1  0.05 0.06 0.  ]
 [0.   0.   1.   0.14 0.08 0.09 0.  ]
 [0.35 1.   0.98 0.21 0.12 0.11 0.  ]
 [0.07 0.   0.53 0.11 0.02 0.01 0.25]
 [0.   0.   0.4  0.05 0.01 0.01 0.07]
 [0.07 1.   0.71 0.19 0.08 0.09 0.1 ]
 [0.   0.   0.98 0.22 0.14 0.19 0.  ]
 [0.12 0.   0.79 0.04 0.04 0.04 0.02]
 [0.   0.   0.96 0.17 0.44 0.1  0.  ]
 [0.03 0.   0.76 0.05 0.09 0.04 0.08]
 [0.22 1.   0.95 0.19 0.15 0.09 0.01]
 [0.62 0.   1.   0.28 0.22 0.21 1.  ]
 [0.18 1.   1.   0.27 0.21 0.22 0.  ]
 [0.45 0.   1.   0.16 0.15 0.15 0.04]
 [0.17 0.   0.99 0.09 0.13 0.1  0.04]
 [0.   0.   0.99 0.19 0.16 0.14 0.  ]
 [0.02 1.   0.91 0.16 0.11 0.05 0.02]
 [0.05 1.   0.89 0.37 0.14 0.1  0.2 ]
 [0.57 1.   1.   1.   1.   1.   0.  ]
 [0.33 0.   0.11 0.   0.   0.   0.  ]
 [0.22 1.   0.66 0.94 0.07 0.04 0.02]
 [0.02 0.   0.46 0.04 0.08 0.02 0.  ]
 [0.32 1.   

In [16]:
import math
import random

random.seed(0)


def rand(a, b):
    return (b - a) * random.random() + a


def make_matrix(m, n, fill=0.0):
    mat = []
    for i in range(m):
        mat.append([fill] * n)
    return mat


def sigmoid(x):
    return 1.0 / (1.0 + math.exp(-x))


def sigmoid_derivative(x):
    return x * (1 - x)


class BPNeuralNetwork:
    def __init__(self):
        self.input_n = 0
        self.hidden_n = 0
        self.output_n = 0
        self.input_cells = []
        self.hidden_cells = []
        self.output_cells = []
        self.input_weights = []
        self.output_weights = []
        self.input_correction = []
        self.output_correction = []

    def setup(self, ni, nh, no):
        self.input_n = ni + 1
        self.hidden_n = nh
        self.output_n = no
        # init cells
        self.input_cells = [1.0] * self.input_n
        self.hidden_cells = [1.0] * self.hidden_n
        self.output_cells = [1.0] * self.output_n
        # init weights
        self.input_weights = make_matrix(self.input_n, self.hidden_n)
        self.output_weights = make_matrix(self.hidden_n, self.output_n)
        # random activate
        for i in range(self.input_n):
            for h in range(self.hidden_n):
                self.input_weights[i][h] = rand(-0.2, 0.2)
        for h in range(self.hidden_n):
            for o in range(self.output_n):
                self.output_weights[h][o] = rand(-2.0, 2.0)
        # init correction matrix
        self.input_correction = make_matrix(self.input_n, self.hidden_n)
        self.output_correction = make_matrix(self.hidden_n, self.output_n)

    def predict(self, inputs):
        # activate input layer
        for i in range(self.input_n - 1):
            self.input_cells[i] = inputs[i]
        # activate hidden layer
        for j in range(self.hidden_n):
            total = 0.0
            for i in range(self.input_n):
                total += self.input_cells[i] * self.input_weights[i][j]
            self.hidden_cells[j] = sigmoid(total)
        # activate output layer
        for k in range(self.output_n):
            total = 0.0
            for j in range(self.hidden_n):
                total += self.hidden_cells[j] * self.output_weights[j][k]
            self.output_cells[k] = sigmoid(total)
        return self.output_cells[:]

    def back_propagate(self, case, label, learn, correct):
        # feed forward
        self.predict(case)
        # get output layer error
        output_deltas = [0.0] * self.output_n
        for o in range(self.output_n):
            error = label[o] - self.output_cells[o]
            output_deltas[o] = sigmoid_derivative(self.output_cells[o]) * error
        # get hidden layer error
        hidden_deltas = [0.0] * self.hidden_n
        for h in range(self.hidden_n):
            error = 0.0
            for o in range(self.output_n):
                error += output_deltas[o] * self.output_weights[h][o]
            hidden_deltas[h] = sigmoid_derivative(self.hidden_cells[h]) * error
        # update output weights
        for h in range(self.hidden_n):
            for o in range(self.output_n):
                change = output_deltas[o] * self.hidden_cells[h]
                self.output_weights[h][o] += learn * change + \
                    correct * self.output_correction[h][o]
                self.output_correction[h][o] = change
        # update input weights
        for i in range(self.input_n):
            for h in range(self.hidden_n):
                change = hidden_deltas[h] * self.input_cells[i]
                self.input_weights[i][h] += learn * change + \
                    correct * self.input_correction[i][h]
                self.input_correction[i][h] = change
        # get global error
        error = 0.0
        for o in range(len(label)):
            error += 0.5 * (label[o] - self.output_cells[o]) ** 2
        return error

    def train(self, cases, labels, limit=10000, learn=0.05, correct=0.1):
        for j in range(limit):
            error = 0.0
            for i in range(len(cases)):
                label = labels[i]
                case = cases[i]
                error += self.back_propagate(case, label, learn, correct)

    def test(self):

        self.setup(2, 5, 1)
        self.train(cases, labels, 10000, 0.05, 0.1)
        
        print("==========================================")
        Xtest_predict=[[0 for col in range(1)] for row in range(len(Xtest))]
        print("测试数据真实结果")
        print(Ytest)
        for i in range(0,len(Xtest)):
            # print(self.predict(Xtest[i]),type(self.predict(Xtest[i])))
            if((self.predict(Xtest[i]))[0]>0.5):
                Xtest_predict[i][0]=0
            else:
                Xtest_predict[i][0]=1
        print("测试数据预测结果")
        print(Xtest_predict)
        # print("对比可得准确率为：100%")
                # 计算准确度
        right_count=0
        sumlen=len(Ytest)
        for i in range(0,len(Ytest)):
            if(Ytest[i]==Xtest_predict[i][0]):
                right_count=right_count+1
        # print(right_count)
        print("准确率："+str(right_count)+"/"+str(sumlen))
        
        print("==========================================")
        print("测试数据[0.12,0, 0.72,0.01,0.02,0.02,0]")    
        # for X in Xtest_extra:
        #     print(self.predict(X))
        Xtest_extra_predict=[[0 for col in range(1)] for row in range(len(Xtest_extra))]
        for i in range(0,len(Xtest_extra)):
            # print(self.predict(Xtest[i]),type(self.predict(Xtest[i])))
            if((self.predict(Xtest_extra[i]))[0]>0.5):
                Xtest_extra_predict[i][0]=1
            else:
                Xtest_extra_predict[i][0]=0
        print(Xtest_extra_predict)
        

        
        if (Xtest_extra_predict[0][0] == 0):
            print("分类结果CUSTOMER_TYPE:3G")
            # 0:yes 1:no
        else:
            print("分类结果CUSTOMER_TYPE:2G")
        print("==========================================")

if __name__ == '__main__':
    nn = BPNeuralNetwork()
    nn.test()


测试数据真实结果
[1 1 0 0 1 1 0 0 0]
测试数据预测结果
[[1], [0], [0], [1], [0], [1], [0], [0], [1]]
准确率：5/9
测试数据[0.12,0, 0.72,0.01,0.02,0.02,0]
[[1]]
分类结果CUSTOMER_TYPE:2G
